In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical

## Step #2 - Prepare Data

In [2]:
seed = 7
np.random.seed(seed)

In [3]:
# Our Dataset.
iris_dataset = pd.read_csv('iris.csv', header=None)
iris_dataset

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
iris_dataset.shape

(150, 5)

In [5]:
iris_dataset.values

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
       [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'],
       [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'],
       [4.8, 3.0, 1.4, 0.1, 'Iris-setosa'],
       [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'],
       [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'],
       [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'],
       [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'],
       [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'],
       [5.7, 3.8, 1.7, 0.3, 'Iris-setosa'],
       [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
       [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'],
       [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'],
       [4.6, 3.6, 1.0, 0.2, 'Iri

In [6]:
X = iris_dataset.iloc[:,0:4].astype('float')
Y = iris_dataset.iloc[:,4]

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)
dummy_y[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

## Step #3 - Baseline model based with single layer.

In [10]:
# define baseline model
def baseline_model():
    # Create model
    model = Sequential()
    model.add(Dense(8,activation='relu',input_shape=(4,)))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

## Step #4 - Evaluate The Model with k-Fold Cross Validation

In [11]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (3.27%)


## Step #5 - Tuning Layers and Number of Neurons in The Model

### Step #5.1 - Evaluate a Smaller Network

In [12]:
def smaller_model():
    # Create model
    model = Sequential()
    model.add(Dense(8,activation='relu',input_shape=(4,)))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [13]:
estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 93.33% (9.89%)


### Step #5.2 - Evaluate a Larger Network

In [14]:
def larger_model():
    # Create model
    model = Sequential()
    model.add(Dense(8,activation='relu',input_shape=(4,)))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [15]:
estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 90.00% (21.34%)


## Step #6. Really Scaling up: developing a model that overfits

In [17]:
def scaled_to_overfit_model():
    # Create model
    model = Sequential()
    model.add(Dense(8,activation='relu',input_shape=(4,)))
    model.add(Dense(8,activation='relu'))  # Add more layers
    model.add(Dense(8,activation='relu'))  # Add more layers
    model.add(Dense(16,activation='relu')) # Made layers bigger
    model.add(Dense(4,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [18]:
estimator = KerasClassifier(build_fn=larger_model, epochs=250, batch_size=5, verbose=0) # Increased Epoch
kfold = KFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 98.00% (4.27%)


## Step #7. Tuning the Model

In [21]:
def tunned_model():
    # Create model
    model = Sequential()
    model.add(Dense(8,activation='relu',input_shape=(4,)))
    model.add(Dense(8,activation='relu'))  # Add more layers
    model.add(Dense(4,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [22]:
estimator = KerasClassifier(build_fn=tunned_model, epochs=200, batch_size=5, verbose=0) # Increased Epoch
kfold = KFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 96.67% (4.47%)


## Step #8. Rewriting the code using the Keras Functional API

In [25]:
def functional_model():
    # Model.
    model = Sequential()
    model.add(Dense(8,activation='relu',input_shape=(4,)))
    model.add(Dense(8,activation='relu'))  # Add more layers
    model.add(Dense(3,activation='softmax'))
    # Compile Model.
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model
#
# -- Training Model
model = functional_model()
model.fit(X,dummy_y, batch_size=5,epochs=200)
    

Train on 150 samples
Epoch 1/200
150/150 [==============================] - 1s 6ms/sample - loss: 0.9835 - acc: 0.6600
Epoch 2/200
150/150 [==============================] - 0s 638us/sample - loss: 0.9270 - acc: 0.6667
Epoch 3/200
150/150 [==============================] - 0s 637us/sample - loss: 0.8612 - acc: 0.6600
Epoch 4/200
150/150 [==============================] - 0s 657us/sample - loss: 0.8073 - acc: 0.6600
Epoch 5/200
150/150 [==============================] - 0s 648us/sample - loss: 0.7658 - acc: 0.6600
Epoch 6/200
150/150 [==============================] - 0s 642us/sample - loss: 0.7293 - acc: 0.6667
Epoch 7/200
150/150 [==============================] - 0s 644us/sample - loss: 0.6923 - acc: 0.6600
Epoch 8/200
150/150 [==============================] - 0s 629us/sample - loss: 0.6587 - acc: 0.6667
Epoch 9/200
150/150 [==============================] - 0s 650us/sample - loss: 0.6230 - acc: 0.6667
Epoch 10/200
150/150 [==============================] - 0s 664us/sample - loss: 0

150/150 [==============================] - 0s 621us/sample - loss: 0.0892 - acc: 0.9800
Epoch 83/200
150/150 [==============================] - 0s 618us/sample - loss: 0.0923 - acc: 0.9533
Epoch 84/200
150/150 [==============================] - 0s 620us/sample - loss: 0.0942 - acc: 0.9733
Epoch 85/200
150/150 [==============================] - 0s 638us/sample - loss: 0.0935 - acc: 0.9800
Epoch 86/200
150/150 [==============================] - 0s 644us/sample - loss: 0.0881 - acc: 0.9733
Epoch 87/200
150/150 [==============================] - 0s 645us/sample - loss: 0.0867 - acc: 0.9733
Epoch 88/200
150/150 [==============================] - 0s 632us/sample - loss: 0.0847 - acc: 0.9800
Epoch 89/200
150/150 [==============================] - 0s 640us/sample - loss: 0.0874 - acc: 0.9733
Epoch 90/200
150/150 [==============================] - 0s 664us/sample - loss: 0.0845 - acc: 0.9667
Epoch 91/200
150/150 [==============================] - 0s 719us/sample - loss: 0.0880 - acc: 0.9800
Epo

## Step #9. Rewriting the code by doing Model Subclassing

In [36]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.dense1 = Dense(8,activation='relu')
        self.dense2 = Dense(8,activation='relu')
        self.dense3 = Dense(3,activation='softmax')
        
    def call(self,inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
def subclass_model():
    inputs = tf.keras.Input(shape=(4,))
    mymodel = MyModel()
    outputs = mymodel.call(inputs)
    # Keras Model.
    model = tf.keras.Model(inputs,outputs)
    # Compile.
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

my_model = subclass_model()
my_model.fit(X,dummy_y,epochs=200,batch_size=5)

Train on 150 samples
Epoch 1/200
150/150 [==============================] - 1s 7ms/sample - loss: 1.4683 - acc: 0.3333
Epoch 2/200
150/150 [==============================] - 0s 719us/sample - loss: 1.1815 - acc: 0.3333
Epoch 3/200
150/150 [==============================] - 0s 689us/sample - loss: 1.0667 - acc: 0.3333
Epoch 4/200
150/150 [==============================] - 0s 678us/sample - loss: 1.0247 - acc: 0.5067
Epoch 5/200
150/150 [==============================] - 0s 665us/sample - loss: 0.9985 - acc: 0.5667
Epoch 6/200
150/150 [==============================] - 0s 649us/sample - loss: 0.9759 - acc: 0.6533
Epoch 7/200
150/150 [==============================] - 0s 660us/sample - loss: 0.9497 - acc: 0.6600
Epoch 8/200
150/150 [==============================] - 0s 626us/sample - loss: 0.9206 - acc: 0.6667
Epoch 9/200
150/150 [==============================] - 0s 663us/sample - loss: 0.8879 - acc: 0.6667
Epoch 10/200
150/150 [==============================] - 0s 640us/sample - loss: 0

150/150 [==============================] - 0s 660us/sample - loss: 0.1017 - acc: 0.9800
Epoch 83/200
150/150 [==============================] - 0s 637us/sample - loss: 0.1040 - acc: 0.9733
Epoch 84/200
150/150 [==============================] - 0s 654us/sample - loss: 0.1003 - acc: 0.9800
Epoch 85/200
150/150 [==============================] - 0s 641us/sample - loss: 0.1012 - acc: 0.9667
Epoch 86/200
150/150 [==============================] - 0s 639us/sample - loss: 0.0970 - acc: 0.9800
Epoch 87/200
150/150 [==============================] - 0s 642us/sample - loss: 0.0999 - acc: 0.9733
Epoch 88/200
150/150 [==============================] - 0s 646us/sample - loss: 0.0971 - acc: 0.9800
Epoch 89/200
150/150 [==============================] - 0s 658us/sample - loss: 0.1002 - acc: 0.9800
Epoch 90/200
150/150 [==============================] - 0s 663us/sample - loss: 0.0969 - acc: 0.9533
Epoch 91/200
150/150 [==============================] - 0s 666us/sample - loss: 0.0962 - acc: 0.9800
Epo

## Step #10. Rewriting the code without using scikit-learn.

In [38]:
def checking_accuracy_model():
    # Model.
    model = Sequential()
    model.add(Dense(8,activation='relu',input_shape=(4,)))
    model.add(Dense(8,activation='relu'))  # Add more layers
    model.add(Dense(3,activation='softmax'))
    # Compile Model.
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# -- Training Model
model = functional_model()
# model_history = model.fit(X,dummy_y, batch_size=5,epochs=200, verbose=0)

# -- K-Fold Cross validation.
estimator = KerasClassifier(build_fn=checking_accuracy_model, epochs=200, batch_size=5, verbose=0) # Increased Epoch
kfold = KFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 97.33% (3.27%)
